In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models

import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
eng_characters = [chr(i) for i in range(ord('a'), ord('z')+1)]  # English lowercase letters
eng_characters += [chr(i) for i in range(ord('A'), ord('Z')+1)]  # English uppercase letters
eng_characters += [chr(i) for i in range(ord('0'), ord('9')+1)]  # Digits
eng_characters += ['?','<','>']
# Add other characters as needed, such as punctuation marks, special symbols, etc.

print(eng_characters)
# Define the range of Unicode code points for Telugu characters
TELUGU_START = 0x0C00
TELUGU_END = 0x0C7F

# Generate all Telugu characters
telugu_characters = [chr(code_point) for code_point in range(TELUGU_START, TELUGU_END + 1)]
telugu_characters += ['?','<','>']

print(telugu_characters)



['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '<', '>']
['ఀ', 'ఁ', 'ం', 'ః', 'ఄ', 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఌ', '\u0c0d', 'ఎ', 'ఏ', 'ఐ', '\u0c11', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'ఙ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', '\u0c29', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 'ళ', 'ఴ', 'వ', 'శ', 'ష', 'స', 'హ', '\u0c3a', '\u0c3b', '\u0c3c', 'ఽ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ౄ', '\u0c45', 'ె', 'ే', 'ై', '\u0c49', 'ొ', 'ో', 'ౌ', '్', '\u0c4e', '\u0c4f', '\u0c50', '\u0c51', '\u0c52', '\u0c53', '\u0c54', 'ౕ', 'ౖ', '\u0c57', 'ౘ', 'ౙ', 'ౚ', '\u0c5b', '\u0c5c', '\u0c5d', '\u0c5e', '\u0c5f', 'ౠ', 'ౡ', 'ౢ', 'ౣ', '\u0c64', '\u0c65', '౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯'

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

train_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_train.csv'
valid_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_valid.csv'
# Load train and validation datasets

# Load the datasets
train_dataset = pd.read_csv(train_file, header=None)
valid_dataset = pd.read_csv(valid_file, header=None)

# Extract the source and target columns
source_data = train_dataset.iloc[:, 0].values
target_data = train_dataset.iloc[:, 1].values


In [4]:
source_data = ['<' + s + '>' for s in source_data]
target_data = ['<' + t + '>' for t in target_data]

In [5]:
loc = 1
e,t = train_dataset.iloc[loc]
char_to_index_eng = {char: i for i, char in enumerate(eng_characters)}
index_to_char_eng = {i: char for char, i in char_to_index_eng.items()}
def char_to_tensor_eng(word, char_to_index_eng):
    return torch.tensor([char_to_index_eng[char] for char in word], dtype=torch.long)

print(e)
print(char_to_tensor_eng(e,char_to_index_eng))

char_to_index_tel = {char: i for i, char in enumerate(telugu_characters)}
index_to_char_tel = {i: char for char, i in char_to_index_tel.items()}
def char_to_tensor_tel(word, char_to_index_tel):
    return torch.tensor([char_to_index_tel[char] for char in word], dtype=torch.long)

print(t)
t1 = char_to_tensor_tel(t,char_to_index_tel)
print(t1)

characters = [telugu_characters[char_int.item()] for char_int in t1]
string = ''.join(characters)
# Print the characters
print("Characters:", string)
# for batch_idx, (data, target) in enumerate(train_loader):
#     print(f"Batch {batch_idx}:")
#     print("Data shape:", data.shape)
#     print("Target shape:", target.shape)
#     # You can print more details about the data and target if needed
#     break  # This will break after printing the first batch

vastadira
tensor([21,  0, 18, 19,  0,  3,  8, 17,  0])
వస్తాదిరా
tensor([53, 56, 77, 36, 62, 38, 63, 48, 62])
Characters: వస్తాదిరా


In [6]:

source = [char_to_tensor_eng(s, char_to_index_eng) for s in source_data]
target = [char_to_tensor_tel(s, char_to_index_tel) for s in target_data]

In [7]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset
# Pad the tensors to the same length
source_padded = pad_sequence(source, batch_first=True, padding_value=62)
target_padded = pad_sequence(target, batch_first=True, padding_value=128)

# Find the maximum length
max_length = max(source_padded.size(1), target_padded.size(1))

# Pad source and target to the same length
source_padded = torch.nn.functional.pad(source_padded, (0, max_length - source_padded.size(1)), value=62)
target_padded = torch.nn.functional.pad(target_padded, (0, max_length - target_padded.size(1)), value=128)

# Add 62 at the beginning and 63 at the end of each item in source_padded
# source_padded = torch.cat((torch.full((source_padded.shape[0], 1), 63, dtype=torch.long), 
#                            source_padded, torch.full((source_padded.shape[0], 1), 64, dtype=torch.long)), dim=1)
# target_padded = torch.cat((torch.full((target_padded.shape[0], 1), 129, dtype=torch.long), 
#                            target_padded, torch.full((target_padded.shape[0], 1), 130, dtype=torch.long)), dim=1)

In [8]:
target_padded[0]

tensor([129,  53,  48,  77,  23,  62,  50,  53,  62,  48,  63,  40,  71, 130,
        128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
        128, 128])

In [9]:
def t2p(s,t):
    characters = [eng_characters[char_int.item()] for char_int in s]
    string1 = ''.join(characters)
#     print(string)
    characters = [telugu_characters[char_int.item()] for char_int in t]
    string2 = ''.join(characters)
    print(f"{string1} {len(string1)} : {string2} {len(string2)} :")
t2p(source_padded[0],target_padded[0])

<vargaalavaarine>????????????? 30 : <వర్గాలవారినే>???????????????? 30 :


In [10]:
# Create a TensorDataset
dataset = TensorDataset(source_padded, target_padded)

# Create a DataLoader
# batch_size = 64
def train_loader(batch_size):
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return train_dataloader



In [11]:
#valid_dataset = pd.read_csv(valid_file, header=None)

# Extract the source and target columns
val_source_data = valid_dataset.iloc[:, 0].values
val_target_data = valid_dataset.iloc[:, 1].values

val_source_data = ['<' + s + '>' for s in val_source_data]
val_target_data = ['<' + t + '>' for t in val_target_data]

val_source = [char_to_tensor_eng(s, char_to_index_eng) for s in val_source_data]
val_target = [char_to_tensor_tel(s, char_to_index_tel) for s in val_target_data]

val_source_padded = pad_sequence(val_source, batch_first=True, padding_value=62)
val_target_padded = pad_sequence(val_target, batch_first=True, padding_value=128)

# Find the maximum length
max_length = max(val_source_padded.size(1), val_target_padded.size(1))

# Pad source and target to the same length
val_source_padded = torch.nn.functional.pad(val_source_padded, (0, max_length - val_source_padded.size(1)), value=62)
val_target_padded = torch.nn.functional.pad(val_target_padded, (0, max_length - val_target_padded.size(1)), value=128)
# Create a TensorDataset
val_dataset = TensorDataset(val_source_padded, val_target_padded)

def val_loader(batch_size):
    valid_dataloader = DataLoader(val_dataset, batch_size=batch_size)
    return valid_dataloader

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, cell_type, batch_size, dropout_in):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.cell_type = cell_type
        self.embeddin_size = embedding_size
        self.dropout_in = dropout_in
        # Character embedding layer
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        # Encoder RNN
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout_in, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_in, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout_in,  batch_first=True)
        
    def forward(self, x, hidden):
        embedded = self.embedding(x)
        # embedded shape: (seq_length:max_length, N(batch size:32), embedding_size:100)
        output, hidden = self.rnn(embedded, hidden)
        return hidden
    
    def getInitialState(self):
        return torch.zeros(self.num_layers,self.batch_size,self.hidden_size).to(device)


class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, 
                 output_size, num_layers, cell_type, dropout_in):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.embeddin_size = embedding_size
        self.dropout_in = dropout_in
        
        # Devanagari character embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)
        
        # Decoder RNN
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout_in, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_in, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout_in,  batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
    
    def forward(self, x, hidden):
        # shape of x: (N) but required (1, N)
#         x = x.unsqueeze(0)
        
        embedded = self.embedding(x)
        # embedded shape: (1, N, embedding_size) #n * 1 * embd
        output, hidden = self.rnn(embedded, hidden)  
        # shape of outputs: (N*1*131)
        predictions = self.softmax(self.fc(output))
        # shape of predictionos:  (1, N, length of tel_char)
#         predictions = predictions.squeeze(0)

        
        return predictions, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
    
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.shape[0]
        target_len = target.shape[1]
        target_letter_size = 131
#         print(source.shape)
#         print(target.shape)
        
        outputs = torch.zeros(target_len, batch_size, target_letter_size).to(device)
        
        # Encoder forward pass
        if(self.cell_type == "LSTM"):
            hidden = (self.encoder.getInitialState(), self.encoder.getInitialState())
        else:
            hidden = self.encoder.getInitialState()
#         hidden.to(device)
        hidden = self.encoder(source, hidden)
        
#         print(hidden.shape)
        x = target[:,0].view(batch_size, 1)
#         print( "target :",x.shape)
#         print(x)
        
#         # Decoder initial hidden state
#         decoder_hidden = encoder_hidden        
#         # Decoder input (start token)
#         decoder_input = torch.tensor([[SOS_token]] * batch_size)      
        # Decoder forward pass
    
        for t in range(0, target_len):
            output, hidden = self.decoder(x, hidden)
            output = output[:, -1, :]
            outputs[t] = output
            best_guess = output.argmax(1)
            best_guess = best_guess.view(batch_size, 1) 
            x = target[:,t].view(batch_size,1) if np.random.rand() < teacher_forcing_ratio else best_guess
        
        return outputs


In [13]:
input_size = len(char_to_index_eng)
print(input_size)
char_to_index_eng['?']

65


62

In [14]:
# Define the model architecture
input_size = len(char_to_index_eng)
output_size = len(char_to_index_tel)
# hidden_size = 256
# embedding_size = 256
# num_layers = 1
# cell_type = 'LSTM'
# LEARNING_RATE = 0.0001
# NUM_EPOCHS = 10

def train(hidden_size, embedding_size, 
          num_layers, cell_type, LEARNING_RATE, NUM_EPOCHS, batch_size, optimizer_in, dropout_in):

    train_dataloader = train_loader(batch_size)
    valid_dataloader = val_loader(batch_size)
    encoder_net = Encoder(input_size, embedding_size, hidden_size, num_layers, cell_type, batch_size, dropout_in).to(device)
    decoder_net = Decoder(output_size, embedding_size, hidden_size, 
                     output_size, num_layers, cell_type, dropout_in).to(device)
    model = Seq2Seq(encoder_net, decoder_net, cell_type).to(device)

    criterion = nn.NLLLoss()
    if(optimizer_in == 'adam'):
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    else:
        optimizer = optim.NAdam(model.parameters(), lr=LEARNING_RATE)

    step = 0
    # Train the model
    for epoch in range(NUM_EPOCHS):
        model.train()
        total_loss = 0
        total_correct = 0
        accuracy = 0
        for batchid,(inp_data,target) in enumerate(tqdm(train_dataloader)):
            inp_data = inp_data.to(device)
            target = target.to(device)

            output = model(inp_data, target)
            indices = output.argmax(2)

            if(batchid==0):
                t2p(inp_data[0],target[0])
                t2p(inp_data[0],indices[:,0])

            optimizer.zero_grad()
            loss = 0
            for i in range(output.shape[0]):
                loss += criterion(output[i], target[:,i])
                total_correct += ((indices[:, i] == target[i]).sum().item() == len(target[i]))
            total_loss += loss.item()/ len(inp_data[0])
            #accuracy += total_correct/batch_size
            # Backpropagation
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1)
            optimizer.step()
            step+=1

        train_loss = total_loss / len(train_dataloader)
        train_accuracy = total_correct/ len(train_dataloader.dataset)
        validation_loss, validation_accuracy = calculate_validation_loss(model, criterion, valid_dataloader, batch_size)
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")
        print("train_acc", train_accuracy)
        print("train_loss", train_loss)
        print("valid_acc", validation_accuracy)
        print("valid_loss", validation_loss)
        
        wandb.log({'train_accuracy':train_accuracy})
        wandb.log({'train_loss':train_loss})
        wandb.log({'val_accuracy':validation_accuracy})
        wandb.log({'val_loss':validation_loss})


In [15]:
def calculate_validation_loss(model, criterion, valid_dataloader,batch_size):
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    accuracy = 0
    total_correct = 0
    step = 0
    with torch.no_grad():
        for batchid,(inp_data,target) in enumerate(tqdm(valid_dataloader)):
            inp_data = inp_data.to(device)
            target = target.to(device)

            output = model(inp_data, target)
            indices = output.argmax(2)
            output_dim = output.shape[-1]
            if(batchid==0):
                t2p(inp_data[0],target[0])
                t2p(inp_data[0],indices[:,0])
                print((indices[:,0] == target[0]).sum().item())

            # Flatten the output and target tensors to compute the loss
            val_loss = 0
            
            for i in range(output.shape[0]):
                val_loss += criterion(output[i], target[:, i])
                total_correct += ((indices[:, i] == target[i]).sum().item() == len(target[i]))
            step +=1
            total_val_loss += val_loss.item() / len(inp_data[0])
            #accuracy += total_correct/batch_size

    avg_val_loss = total_val_loss / len(valid_dataloader)
    accuracy = total_correct/len(valid_dataloader.dataset) #/ len(valid_dataloader)
    return avg_val_loss, accuracy


In [16]:
# train(hidden_size, embedding_size, 
#           num_layers, cell_type, LEARNING_RATE, NUM_EPOCHS)

In [17]:
!pip install wandb
import wandb
wandb.login(key='4823cbdc22dd85ccb0f95f99ade210f09cb97abc')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
def main_fun():
    wandb.init(project ='Assignment3')
    params = wandb.config
    with wandb.init(project = 'Assignment3', name='epochs_'+str(params.num_epochs) 
                    +'layers_'+str(params.num_layers) + 'embedding_'+str(params.embedding_size)
                    +'hidden_size_'+str(params.hidden_size) + 'cell_type_'+str(params.cell_type)
                    +'lear_rate_'+str(params.learning_rate) + 'batch_size'+str(params.batch_size)
                    + 'optim_'+str(params.optimizer) + 'dropout_'+str(params.dropout_in)) as run:
        train(params.hidden_size,params.embedding_size,
              params.num_layers, params.cell_type, params.learning_rate, 
              params.num_epochs, params.batch_size,params.optimizer, params.dropout_in)

sweep_params = {
    'method' : 'bayes',
    'name'   : 'cs23m005',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
            'num_epochs':{'values':[20,25,30]},
            'learning_rate' :{'values':[1e-4, 1e-3]},
            'num_layers' :{'values':[1,2,3,4,5]},
            'embedding_size' :{'values':[128,256]},
            'hidden_size' :{'values':[128,256]},
            'cell_type' :{'values':['LSTM', 'GRU', 'RNN']},
            'batch_size' :{'values':[64,32,128]},
            'optimizer' :{'values':['adam','nadam']},
            'dropout_in' :{'values':[0.2,0.3,0.5]}
            
    }
}
sweepId = wandb.sweep(sweep_params,project = 'Assignment3')
wandb.agent(sweepId,function =main_fun,count = 30)
wandb.finish()

Create sweep with ID: tz8yfuqh
Sweep URL: https://wandb.ai/cs23m005/Assignment3/sweeps/tz8yfuqh


wandb: Agent Starting Run: 4te0hyvo with config:
wandb: 	batch_size: 64
wandb: 	cell_type: GRU
wandb: 	dropout_in: 0.5
wandb: 	embedding_size: 128
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 20
wandb: 	num_layers: 1
wandb: 	optimizer: adam
wandb: Currently logged in as: cs23m005. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240516_112449-4te0hyvo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run vibrant-sweep-1
wandb: ⭐️ View project at https://wandb.ai/cs23m005/Assignment3
wandb: 🧹 View sweep at https://wandb.ai/cs23m005/Assignment3/sweeps/tz8yfuqh
wandb: 🚀 View run at https://wandb.ai/cs23m005/Assignment3/runs/4te0hyvo
wandb: WARNING Ignored wandb.init() ar

<gostani>????????????????????? 30 : <గోస్తాని>???????????????????? 30 :
<gostani>????????????????????? 30 : ౦ఋ౧౴౶ౣౣ౐య౧౻ో఩ఝ౻౻౻఻ైఴ఻౉ౕౕౕౕౕౕౕౕ 30 :


 11%|█         | 7/64 [00:00<00:00, 61.19it/s]

<bheeshmudini>???????????????? 30 : <భీష్ముడిని>?????????????????? 30 :
<bheeshmudini>???????????????? 30 : <ప్ర్ర్ను>?>?????????????????? 30 :
21


...

  9%|▉         | 6/64 [00:00<00:01, 53.68it/s]

<bheeshmudini>???????????????? 30 : <భీష్ముడిని>?????????????????? 30 :
<bheeshmudini>???????????????? 30 : <పావారించని>?????????????????? 30 :
22


100%|██████████| 64/64 [00:01<00:00, 56.86it/s]


Epoch 30, Loss: 1.0148049906094858
train_acc 0.0
train_loss 1.0148049906094858
valid_acc 0.0
valid_loss 0.8192551970481871


wandb:                                                                                
wandb: 
wandb: Run history:
wandb: train_accuracy ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁
wandb:     train_loss █▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁
wandb:   val_accuracy ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:       val_loss █▇▇▇▆▆▆▅▆▆▆▆▆▆▆▆▄▃▂▂▂▂▁▂▂▂▁▁▁▁
wandb: 
wandb: Run summary:
wandb: train_accuracy 0.0
wandb:     train_loss 1.0148
wandb:   val_accuracy 0.0
wandb:       val_loss 0.81926
wandb: 
wandb: 🚀 View run denim-sweep-30 at: https://wandb.ai/cs23m005/Assignment3/runs/ya74i645
wandb: ⭐️ View project at: https://wandb.ai/cs23m005/Assignment3
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240516_224007-ya74i645/logs
